In [1]:
# pip install splinter

In [2]:
# Import Splinter and BeautifulSoup
from splinter import Browser
from bs4 import BeautifulSoup


In [3]:
# Path to chromedriver
# !which chromedriver

In [4]:
# Set the executable path and initialize the chrome browser in splinter
executable_path = {'executable_path': 'c:/Users/indup/bin/chromedriver'}
browser = Browser('chrome',headless=True, **executable_path)

# selenium
# selenium_driver = browser.driver


## Visit the NASA mars news site

In [5]:
# Visit the mars nasa news site
url = 'https://mars.nasa.gov/news/'
browser.visit(url)

# Optional delay for loading the page
browser.is_element_present_by_css("ul.item_list li.slide", wait_time=1)

True

In [6]:
# Convert the browser html to a soup object 
html = browser.html
news_soup = BeautifulSoup(html, 'html.parser')
slide_elem = news_soup.select_one('ul.item_list li.slide')

In [7]:
slide_elem.find("div", class_='content_title')

<div class="content_title"><a href="/news/8556/nasa-updates-mars-2020-mission-environmental-review/" target="_self">NASA Updates Mars 2020 Mission Environmental Review</a></div>

In [8]:
# Use the parent element to find the first a tag and save it as `news_title`
news_title = slide_elem.find("div", class_='content_title').get_text()
news_title

'NASA Updates Mars 2020 Mission Environmental Review'

In [9]:
# Use the parent element to find the paragraph text
news_p = slide_elem.find('div', class_="article_teaser_body").get_text()
news_p

'NASA and the Department of Energy have completed a more detailed risk analysis for the Mars 2020 rover launch from Florida.'

## JPL Space Images Featured Image

In [10]:
# Visit URL
url = 'https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars'
browser.visit(url)

In [11]:
# Find and click the full image button
full_image_elem = browser.find_by_id('full_image')
full_image_elem.click()

In [12]:
# Find the more info button and click that
browser.is_element_present_by_text('more info', wait_time=1)
more_info_elem = browser.find_link_by_partial_text('more info')
more_info_elem.click()

In [13]:
# Parse the resulting html with soup
html = browser.html
img_soup = BeautifulSoup(html, 'html.parser')

In [14]:
# find the relative image url
img_url_rel = img_soup.select_one('figure.lede a img').get("src")
img_url_rel

'/spaceimages/images/largesize/PIA15283_hires.jpg'

In [15]:
# Use the base url to create an absolute url
img_url = f'https://www.jpl.nasa.gov{img_url_rel}'
img_url

'https://www.jpl.nasa.gov/spaceimages/images/largesize/PIA15283_hires.jpg'

## Mars Weather

In [16]:
url = 'https://twitter.com/marswxreport?lang=en'
browser.visit(url)

In [17]:
html = browser.html
weather_soup = BeautifulSoup(html, 'html.parser')

In [18]:
# First, find a tweet with the data-name `Mars Weather`
mars_weather_tweet = weather_soup.find('div', attrs={"class": "tweet", "data-name": "Mars Weather"})

In [19]:
# Next, search within the tweet for the p tag containing the tweet text
mars_weather = mars_weather_tweet.find('p', 'tweet-text').get_text()
mars_weather

'InSight sol 352 (2019-11-23) low -100.7ºC (-149.3ºF) high -22.0ºC (-7.6ºF)\nwinds from the S at 5.7 m/s (12.8 mph) gusting to 21.9 m/s (49.0 mph)\npressure at 6.80 hPapic.twitter.com/fXsaTX3DDB'

## Mars Hemispheres

In [20]:
url = 'https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars'
browser.visit(url)

In [21]:
html = browser.html
hemisphere_soup = BeautifulSoup(html, 'html.parser')

In [22]:
hemisphere_image_urls = []

# First, get a list of all of the hemispheres
links = browser.find_by_css("a.product-item h3")
             
# Next, loop through those links, click the link, find the sample anchor, return the href
for i in range(len(links)):
 
    hemisphere = {}
    
    # We have to find the elements on each loop to avoid a stale element exception
    browser.find_by_css("a.product-item h3")[i].click()
 
    
    # Next, we find the Sample image anchor tag and extract the href
    sample_elem = browser.find_link_by_text('Sample').first
    hemisphere['img_url'] = sample_elem['href']
    
    # Get Hemisphere title
    hemisphere['title'] = browser.find_by_css('h2.title').text

   # Append hemisphere object to list
    hemisphere_image_urls.append(hemisphere)
    
    # Finally, we navigate backwards
    browser.back()
    

In [23]:
hemisphere_image_urls

[{'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif/full.jpg',
  'title': 'Cerberus Hemisphere Enhanced'},
 {'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/schiaparelli_enhanced.tif/full.jpg',
  'title': 'Schiaparelli Hemisphere Enhanced'},
 {'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/syrtis_major_enhanced.tif/full.jpg',
  'title': 'Syrtis Major Hemisphere Enhanced'},
 {'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/valles_marineris_enhanced.tif/full.jpg',
  'title': 'Valles Marineris Hemisphere Enhanced'}]

## Mars Facts

In [29]:
import pandas as pd
df = pd.read_html('https://space-facts.com/mars/')[0]
df.columns=['description', 'value']
df.set_index(['description','value'], inplace = True)
df.index.name=None
df

,
description,value
Equatorial Diameter:,"6,792 km"
Polar Diameter:,"6,752 km"
Mass:,6.39 × 10^23 kg (0.11 Earths)
Moons:,2 (Phobos & Deimos)
Orbit Distance:,"227,943,824 km (1.38 AU)"
Orbit Period:,687 days (1.9 years)
Surface Temperature:,-87 to -5 °C
First Record:,2nd millennium BC
Recorded By:,Egyptian astronomers


In [30]:
# df.to_html()
df.to_html(classes=['table','text-white'], header = False)

'<table border="1" class="dataframe table text-white">\n  <thead>\n    <tr>\n      <th>description</th>\n      <th>value</th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <th>Equatorial Diameter:</th>\n      <th>6,792 km</th>\n    </tr>\n    <tr>\n      <th>Polar Diameter:</th>\n      <th>6,752 km</th>\n    </tr>\n    <tr>\n      <th>Mass:</th>\n      <th>6.39 × 10^23 kg (0.11 Earths)</th>\n    </tr>\n    <tr>\n      <th>Moons:</th>\n      <th>2 (Phobos &amp; Deimos)</th>\n    </tr>\n    <tr>\n      <th>Orbit Distance:</th>\n      <th>227,943,824 km (1.38 AU)</th>\n    </tr>\n    <tr>\n      <th>Orbit Period:</th>\n      <th>687 days (1.9 years)</th>\n    </tr>\n    <tr>\n      <th>Surface Temperature:</th>\n      <th>-87 to -5 °C</th>\n    </tr>\n    <tr>\n      <th>First Record:</th>\n      <th>2nd millennium BC</th>\n    </tr>\n    <tr>\n      <th>Recorded By:</th>\n      <th>Egyptian astronomers</th>\n    </tr>\n  </tbody>\n</table>'

In [26]:
browser.quit()